These functions will compute [BLEU, METEOR, ROUGE, and CIDEr](https://encord.com/blog/vision-language-models-guide/#h3) scores for a list of predicted captions and their corresponding list of ground truth captions.

In [1]:
!pip install nltk rouge-score
!git clone https://github.com/salaniz/pycocoevalcap
%cd pycocoevalcap
!pip install -e .

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=624e9db4f8a9bf8715f9bb8f21ffe65ecfdce4822b45d3f0afd86eec043486d9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Cloning into 'pycocoevalcap'...
remote: Enumerating objects: 821, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 821 (delta 5), reused 19 (delta 4), pack-reused 797
Receiving objects: 100% (821/821), 130.06 MiB | 14.30 MiB/s, done.
Resolving deltas: 100% (424/424), done.
/content/pycocoevalcap
Obtaining file:///content/pycocoevalcap
  Preparing metadata (setup.py) ... done
  Running setup.py develop for pycocoevalcap


In [2]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from pycocoevalcap.cider.cider import Cider

# To download the necessary datasets for nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [34]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # This is required for word_tokenize

def calculate_bleu(references, hypotheses):
    """
    Calculate BLEU score between actual and predicted sentences using smoothing.
    """
    smoothie = SmoothingFunction().method1  # Using method1 for example
    references_tokenized = [[word_tokenize(ref)] for ref in references]
    hypotheses_tokenized = [word_tokenize(hyp) for hyp in hypotheses]
    return corpus_bleu(references_tokenized, hypotheses_tokenized, smoothing_function=smoothie)

def calculate_meteor(references, hypotheses):
    """
    Calculate METEOR score between actual and predicted sentences.
    """
    references_tokenized = [word_tokenize(ref) for ref in references]
    hypotheses_tokenized = [word_tokenize(hyp) for hyp in hypotheses]
    return sum(meteor_score([ref], hyp) for ref, hyp in zip(references_tokenized, hypotheses_tokenized)) / len(hypotheses_tokenized)

def calculate_rouge(references, hypotheses):
    """
    Calculate ROUGE scores between actual and predicted sentences.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {key: 0.0 for key in ['rouge1', 'rouge2', 'rougeL']}
    for ref, hyp in zip(references, hypotheses):
        score = scorer.score(ref, hyp)
        for key in scores:
            scores[key] += score[key].fmeasure
    for key in scores:
        scores[key] /= len(hypotheses)
    return scores

def calculate_cider(references, hypotheses):
    """
    Calculate CIDEr score between actual and predicted sentences.
    """
    scorer = Cider()
    # Convert to the format expected by CIDEr scorer
    hypo_dict = {i: [hyp] for i, hyp in enumerate(hypotheses)}
    ref_dict = {i: [ref] for i, ref in enumerate(references)}
    score, _ = scorer.compute_score(ref_dict, hypo_dict)
    return score

# # Example data
# references = [
#     "A dog sitting on a park bench.",
#     "Two people walking down the street."
# ]
# hypotheses = [
#     "A dog is on a bench in the park.",
#     "Two persons are walking on the road."
# ]

# # Compute scores
# bleu_score = calculate_bleu(references, hypotheses)
# meteor_score = calculate_meteor(references, hypotheses)
# rouge_scores = calculate_rouge(references, hypotheses)
# cider_score = calculate_cider(references, hypotheses)

# print("BLEU Score:", bleu_score)
# print("METEOR Score:", meteor_score)
# print("ROUGE Scores:", rouge_scores)
# print("CIDEr Score:", cider_score)

BLEU Score: 0.04617747988825104
METEOR Score: 0.48989932627121613
ROUGE Scores: {'rouge1': 0.6057692307692307, 'rouge2': 0.14285714285714288, 'rougeL': 0.5432692307692308}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import os
os.getcwd()
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [6]:
img_dirname = "/content/drive/MyDrive/UMich/LLM/Final_Project/imgs/"

invalid_images = []
with open(img_dirname+'invalid_images.txt', 'r') as infile:
    for line in infile:
        invalid_images.append(int(line.strip()))
img_labels = pd.read_csv("/content/drive/MyDrive/UMich/LLM/Final_Project/Contrastive.csv", nrows= 6000)#+len(invalid_images))
img_labels = img_labels[~img_labels.index.isin(invalid_images)]

In [7]:
val_data=img_labels[3721:]
print(len(val_data))

749


In [8]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets
!pip install httpx
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [49]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
#model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16)
model = Blip2ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/UMich/LLM/Final_Project/lora_model', load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from torchvision.io import read_image

label_sentences=[]
predict_sentences=[]
for idx, i in val_data.iterrows():
  print(idx)
  label_sentences.append(i["utterance"])
  emotion=i["emotion"]
  #prompt = f"Q: Give me a descriptive caption that evokes a feeling of {emotion} A:"
  prompt = f"This is a caption that invokes {emotion}:"
  painting_name = i["painting"]+ '.jpeg'
  painting = os.path.join(img_dirname, painting_name)
  image = read_image(painting)
  inputs = processor(images=image, text=prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)
  generated_ids = model.generate(**inputs, max_new_tokens=41)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
  predict_sentences.append(generated_text)

  if idx==5050:
    break



5000


Both `max_new_tokens` (=41) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5001


Both `max_new_tokens` (=41) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5002


...

Both `max_new_tokens` (=41) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5049


Both `max_new_tokens` (=41) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5050


Both `max_new_tokens` (=41) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [51]:
print(label_sentences)
print(predict_sentences)
print(len(label_sentences))
print(len(predict_sentences))

['She is rolling her eyes upward like a typical child or teenager.', 'The woman is hunched over and seems uncomfortable or distressed, and the man is too close, with a pressuring gaze. The lighting is dim and gloomy - the woman cannot see her work well.']
['The girl looks angry and the background is dark.', 'The woman looks like she is in pain and the man is looking at her with concern.']
2
2


In [41]:
bleu_score = calculate_bleu(label_sentences, predict_sentences)
#meteor_score = calculate_meteor(label_sentences[:2], predict_sentences[:2])
rouge_scores = calculate_rouge(label_sentences, predict_sentences)
cider_score = calculate_cider(label_sentences, predict_sentences)

print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score)
print("ROUGE Scores:", rouge_scores)
print("CIDEr Score:", cider_score)

BLEU Score: 0.021786448788507443
METEOR Score: 0.48989932627121613
ROUGE Scores: {'rouge1': 0.20172228466119543, 'rouge2': 0.048552705455600476, 'rougeL': 0.17631096051899728}
CIDEr Score: 0.05332526672514909
